In [1]:
# importing libs

import pandas as pd
import numpy as np
import torch
import evaluate
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding)

from sklearn.model_selection import train_test_split




c:\Users\ASUS\Desktop\Research_Material\p2_model\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# check gpu

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()

Using device: cuda
GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
# load data

# Option 1: Read directly from .xlsx (preserves Bengali text)
df = pd.read_excel("dataset.xlsx")

print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
df.head()

Dataset shape: (4897, 2)
Columns: ['posts', 'labels']


,posts,labels
0,কিভাবে হতাশা থেকে রক্ষা পাওয়া যাবে,2
1,"মাথা সঠিক কাজ করে না, মানসিক বিষন্বতা, দুঃসচিন...",3
2,আমি বেশ কিছুদিন যাবৎ অনেক ডিপ্রেসন এ আছি। আমার...,2
3,কোনো ভাবেই ডিপ্রেশন থেকে বেরিয়ে আসতে পারতেছি না,2
4,আমার বয়স ৩৪ বছর।জীবনে সবকিছু আমার মত করে পেতে ...,2
